In [2]:
import pandas as pd
import pathlib
import os
import re

# 1) Get all participants from respective datasets

In [28]:
# for both sheets in excel
df = pd.read_excel("data/english-results-data.xlsx", sheet_name="Time 1")
df1 = pd.read_excel("data/english-results-data.xlsx", sheet_name="Repeats")

C:\Users\imruh\miniconda3\envs\thesis\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\imruh\miniconda3\envs\thesis\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [29]:
print(df["WAB Type"].unique())
print(df1["WAB Type"].unique())

[nan 'Broca' 'TransMotor' 'not aphasic' 'Conduction' 'Wernicke' 'Anomic'
 'TransMotor ' 'U' 'Anomic ' 'Global' 'TransSensory']
[nan 'Broca' 'Conduction' 'Anomic' 'Wernicke' 'Global' 'TransMotor' 'U'
 'not aphasic' 'TransMotor ' "Broca's" 'not aphasic  ']


In [30]:
def get_participants_files(wab_type, data1, data2):
    data1 = data1.loc[data1["WAB Type"] == wab_type]
    participants = set(data1["Participant ID"])
    data2 = data2.loc[data2["WAB Type"] == wab_type]
    participants1 = set(data2["Participant ID"])
    
    participants.update(participants1)
    ids = list(participants)
    ids.sort(key=str.lower)
    return ids

In [41]:
# number of brocas patients
ids = get_participants_files("Broca", df, df1)
ids

['ACWT01a',
 'ACWT05a',
 'ACWT08a',
 'adler10a',
 'adler11a',
 'adler13a',
 'adler16a',
 'adler19a',
 'adler25a',
 'BU07a',
 'BU08a',
 'cmu02b',
 'elman03a',
 'elman06a',
 'elman08a',
 'elman09a',
 'elman11a',
 'fridriksson-1-1011-1',
 'fridriksson-2-1006-1',
 'fridriksson-2-1006-2',
 'fridriksson-2-1006-3',
 'fridriksson-2-1006-4',
 'fridriksson-2-1006-5',
 'fridriksson-2-1006-6',
 'fridriksson-2-1006-LARC',
 'fridriksson-2-1008-1',
 'fridriksson-2-1008-2',
 'fridriksson-2-1008-3',
 'fridriksson-2-1008-4',
 'fridriksson-2-1008-5',
 'fridriksson-2-1008-6',
 'fridriksson-2-1008-LARC',
 'fridriksson-2-1010-1',
 'fridriksson-2-1010-2',
 'fridriksson-2-1010-3',
 'fridriksson-2-1010-4',
 'fridriksson-2-1010-5',
 'fridriksson-2-1010-6',
 'fridriksson-2-1010-LARC',
 'fridriksson-2-1012-1',
 'fridriksson-2-1012-2',
 'fridriksson-2-1012-3',
 'fridriksson-2-1012-4',
 'fridriksson-2-1012-5',
 'fridriksson-2-1012-6',
 'fridriksson-2-1012-LARC',
 'fridriksson-2-1015-1',
 'fridriksson-2-1015-2',
 'f

In [20]:
w_ids = get_participants_files("Wernicke", df, df1)
t_ids = get_participants_files("TransSensory", df, df1)
c_ids = get_participants_files("Conduction", df, df1)
a_ids = get_participants_files("Anomic", df, df1)
a1_ids = get_participants_files("Anomic ", df, df1)

In [38]:
w_ids

['ACWT10a',
 'ACWT11a',
 'adler06a',
 'adler23a',
 'BU10a',
 'elman12a',
 'elman14a',
 'fridriksson-2-1003-1',
 'fridriksson-2-1003-2',
 'fridriksson-2-1003-3',
 'fridriksson-2-1003-4',
 'fridriksson-2-1003-5',
 'fridriksson-2-1003-6',
 'fridriksson-2-1003-LARC',
 'fridriksson-2-1047-1',
 'fridriksson-2-1047-2',
 'fridriksson-2-1047-3',
 'fridriksson-2-1047-4',
 'fridriksson-2-1047-5',
 'fridriksson-2-1047-6',
 'fridriksson-2-1047-LARC',
 'fridriksson-2-1089-1',
 'fridriksson-2-1089-2',
 'fridriksson-2-1089-3',
 'fridriksson-2-1089-4',
 'fridriksson-2-1089-5',
 'fridriksson-2-1089-6',
 'fridriksson-2-1089-LARC',
 'fridriksson-2-1116-1',
 'fridriksson-2-1116-2',
 'fridriksson-2-1116-3',
 'fridriksson-2-1116-4',
 'fridriksson-2-1116-5',
 'fridriksson-2-1116-6',
 'garrett01a',
 'kansas05a',
 'kansas12a',
 'kansas14a',
 'kansas23a',
 'kurland01c',
 'kurland01d',
 'kurland16a',
 'kurland16b',
 'kurland18a',
 'kurland18b',
 'scale06d',
 'scale11b',
 'scale24a',
 'scale38a',
 'thompson03a',
 

# 2) Get file names we are interested in for PWA

In [63]:
def get_file_names(wab_type, data1, data2):
    ids = get_participants_files(wab_type, data1, data2)
    brocas = [] # to check for missing data
    filenames = [] # to keep track of file names
    
    lower_ids = [x.lower() for x in ids]
    
    data_dir = str(pathlib.Path().resolve()) + "\\data\\data_pwa\\"
    # print(data_dir)
    
    for subdir, dirs, files in os.walk(data_dir):
        for file in files:
            i = file.replace(".cha","")
            dirname = os.path.basename(subdir).lower()
 
            # each naming convention is different
            if (dirname == "fridriksson-2" or dirname == "umd") and dirname + '-' + i.lower() in lower_ids:
                brocas.append(dirname + '-' + i)
                filenames.append(file)
            
            if i.lower() in lower_ids:
                brocas.append(i)
                filenames.append(file)
            if dirname+i.lower() in lower_ids:
                brocas.append(dirname+i)
                filenames.append(file)
    print(f"Found datafiles: {len(brocas)}")
    print(f"Actual datafiles in excel: {len(ids)}")
    
    lower_brocas= [x.lower() for x in brocas]
    
    print(f"Missing datafiles: {set(lower_ids) - set(lower_brocas)}")
    # missing data
    print(f"Number of missing datafiles: {len(set(lower_ids) - set(lower_brocas))}")
    
    return filenames

In [68]:
WAB = ["Wernicke", "TransSensory", "Conduction", "Anomic", "Anomic "] 
fname = ["wernicke_fname.csv", "transsensory_fname.csv", "conduction_fname.csv", "anomic_fname.csv", "anomic1_fname.csv"]

for type, file in zip(WAB, fname):
    print("WAB: ", type)
    filenames = get_file_names(type, df, df1)
    print("------------------")
    aphasia = pd.DataFrame(filenames).T
    aphasia.to_csv("data/"+file, index=False, header=False)

WAB:  Wernicke
Found datafiles: 47
Actual datafiles in excel: 61
Missing datafiles: {'fridriksson-2-1089-larc', 'fridriksson-2-1047-6', 'fridriksson-2-1047-1', 'fridriksson-2-1047-larc', 'fridriksson-2-1047-2', 'fridriksson-2-1089-6', 'fridriksson-2-1089-3', 'fridriksson-2-1089-1', 'fridriksson-2-1047-4', 'fridriksson-2-1047-3', 'fridriksson-2-1047-5', 'fridriksson-2-1089-5', 'fridriksson-2-1089-2', 'fridriksson-2-1089-4'}
Number of missing datafiles: 14
------------------
WAB:  TransSensory
Found datafiles: 2
Actual datafiles in excel: 2
Missing datafiles: set()
Number of missing datafiles: 0
------------------
WAB:  Conduction
Found datafiles: 99
Actual datafiles in excel: 151
Missing datafiles: {'fridriksson-2-1045-larc', 'fridriksson-2-1041-6', 'fridriksson-2-1060-6', 'fridriksson-2-1045-6', 'fridriksson-2-1110-3', 'neural55-2', 'fridriksson-2-1034-3', 'fridriksson-2-1034-2', 'fridriksson-2-1060-5', 'fridriksson-2-1077-3', 'fridriksson-2-1056-4', 'fridriksson-2-1110-larc', 'fridrik

In [64]:
WAB = "Broca" 
fname = "broca_fname.csv"

filenames = get_file_names(WAB,df, df1)

file_broca = pd.DataFrame(filenames).T
file_broca.to_csv(fname, index=False, header=False)

Found datafiles: 174
Actual datafiles in excel: 326
Missing datafiles: {'fridriksson-2-1101-4', 'fridriksson-2-1095-5', 'fridriksson-2-1010-4', 'fridriksson-2-1012-2', 'fridriksson-2-1053-larc', 'fridriksson-2-1063-larc', 'fridriksson-2-1053-6', 'fridriksson-2-1036-3', 'fridriksson-2-1053-2', 'fridriksson-2-1064-1', 'fridriksson-2-1101-larc', 'fridriksson-2-1106-6', 'fridriksson-2-1064-2', 'fridriksson-2-1067-4', 'fridriksson-2-1067-6', 'fridriksson-2-1064-6', 'fridriksson-2-1062-2', 'fridriksson-2-1010-5', 'fridriksson-2-1095-3', 'fridriksson-2-1101-3', 'fridriksson-2-1112-larc', 'fridriksson-2-1080-1', 'fridriksson-2-1096-6', 'fridriksson-2-1010-3', 'fridriksson-2-1010-6', 'fridriksson-2-1039-3', 'fridriksson-2-1010-2', 'fridriksson-2-1095-larc', 'fridriksson-2-1070-6', 'fridriksson-2-1070-3', 'fridriksson-2-1080-3', 'fridriksson-1-1011-1', 'fridriksson-2-1036-2', 'umd-mma21a', 'fridriksson-2-1062-4', 'fridriksson-2-1072-larc', 'fridriksson-2-1106-1', 'fridriksson-2-1067-3', 'fridrik

# 3) Getting the data preprocessed
#### Run setup_dataframe.py and then preprocessing_edited.py

# 4) Section the preprocessed data into scenarios (merge sentences) and combine broca data and control data

In [20]:
broca_fname = "data_broca_preprocessed.csv"
control_fname = "data_control_preprocessed.csv"

broca_data = pd.read_csv(broca_fname).dropna()
control_data = pd.read_csv(control_fname, encoding='utf8').dropna()

FileNotFoundError: [Errno 2] No such file or directory: 'data_broca_preprocessed.csv'

In [ ]:
broca_patients = len(broca_data["source_file"].unique())
control = len(control_data["source_file"].unique())
print(f"Brocas patients: {broca_patients}, control: {control}")

In [ ]:
def merge_scenario(data, label):
    columns = ['source_file', 'scenario', 'text', 'label']
    new_data = [] # pd.DataFrame(columns=columns)
    text = ""

    for index, row in data.iterrows():
        # print(f"index : {index}, scenario {row['scenario']}")
        if index == 0:
            prev_scenario = row["scenario"] 
            prev_sourcefile = row["source_file"]
        current_scenario = row["scenario"]
        current_sourcefile = row["source_file"]
        
        if current_scenario == prev_scenario and current_sourcefile == prev_sourcefile:
            # print("same scenario")
            # print(text)
            # print()
            text += " " + row["preprocessed_text"]

        else:
            # print("new scenario")
            # print(text)
            # print()
            new_data.append([row["source_file"], prev_scenario, re.sub(' +', ' ', text), label])
            text = ""
        
        prev_scenario = current_scenario
        prev_sourcefile = current_sourcefile
        
        # if index == 74:
        #     break
    
    return pd.DataFrame(new_data, columns=columns)

In [ ]:
# LABEL 1 FOR BROCA 0 FOR NOT APHASIC, for classifier
broca_data_merge = merge_scenario(broca_data, 1)
control_data_merge = merge_scenario(control_data, 0)
data_full_scenario = pd.concat([broca_data_merge, control_data_merge], ignore_index=True)

In [ ]:
# A bit imbalanced?
print(len(broca_data_merge))
print(len(control_data_merge))

In [ ]:
# shuffle data
data_full_scenario = data_full_scenario.sample(frac=1).reset_index(drop=True)

In [ ]:
data_full_scenario

In [ ]:
data_full_scenario.to_csv("all_data_processed.csv")

# 5) Alternative: don't merge scenarios

In [73]:
import pandas as pd
broca_fname = "data/preprocessed_broca.csv"
control_fname = "data/preprocessed_control.csv"

broca_data = pd.read_csv(broca_fname).dropna()
control_data = pd.read_csv(control_fname, encoding='utf8').dropna()

print(broca_data.columns)
broca_data["label"] = 1
control_data["label"] = 0

data_full_scenario = pd.concat([broca_data, control_data], ignore_index=True)
data_full_scenario = data_full_scenario.sample(frac=1).reset_index(drop=True)
data_full_scenario["preprocessed_text"] = [x.lstrip().rstrip().lower() for x in data_full_scenario['preprocessed_text']]

data_full_scenario.to_csv("data/processed_merge.csv")

Index(['Unnamed: 0', 'scenario', 'source_file', 'original_text',
       'preprocessed_text'],
      dtype='object')
